<a href="https://colab.research.google.com/github/grkidwell/Integrated_magnetic/blob/master/48V_integratedmagnetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Integrated Magnetic

### Libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt

###Classes

#### Core

In [0]:
class PQcore:
    def __init__(self, core_geom, u, gaps):
        self.core_geometry = core_geom
        self.gap_lengths = gap_lengths
        self.u = u
        self.area_cp2l_ratio=2
        
        self.leg_areas = {'left':self.core_geometry['Area']/self.area_cp2l_ratio,
                          'right':self.core_geometry['Area']/self.area_cp2l_ratio,
                          'center':self.core_geometry['Area']}
        
        self.segment_areas={
            'AB':self.leg_areas['left'],
            'BC':self.leg_areas['right'],
            'CD':self.leg_areas['right'], 
            'DE':self.leg_areas['right'],
            'EF':self.leg_areas['left'],
            'AF':self.leg_areas['left'],
            'BE':self.leg_areas['center']}
        
        self.segment_fluxpath_lengths={
            'AB':self.core_geometry['length']/2,
            'BC':self.core_geometry['length']/2,
            'CD':self.core_geometry['height'] - self.gap_lengths['CD'],
            'DE':self.core_geometry['length']/2,
            'EF':self.core_geometry['length']/2,
            'AF':self.core_geometry['height'] - self.gap_lengths['AF'],
            'BE':self.core_geometry['height'] - self.gap_lengths['BE']}

#### Winding

In [0]:
class Winding:
    def __init__(self, turns):
        self.turns=turns

#### Reluctance

In [0]:
class Reluctance_model:

  #valid for 3 leg EE/EI type core geometry (EE, EI, PQ, RM, etc.)
  
    def __init__(self,core):
      
        self.core=core
        
        #create dictionary of core segment reluctances
        self.segment_reluctances = {key: self.reluctance(ur=self.core.u, length=value, area=self.core.segment_areas[key])  for key, value in self.core.segment_fluxpath_lengths.items()}
    
        #add gap reluctances to legs
        for key, value in self.core.gap_lengths.items():
            self.segment_reluctances[key]+=self.reluctance(ur=1, length=value, area=self.core.segment_areas[key])
        
        #each leg is made up of segments
        self.leg_segments={
            'left':   ['AB','EF','AF'],
            'right':  ['BC','CD','DE'],
            'center': ['BE']
        }
        
        #create dictionary of leg reluctances
        self.leg_reluctances = {leg: self.total_reluctance(segmentlist) for leg, segmentlist in self.leg_segments.items()}
        
        #create dictionary of reluctances seen by each winding
        self.winding_reluctances = {'half_window1': self.leg_reluctances['left']  + self.parallel(self.leg_reluctances['center'],self.leg_reluctances['right']),
                                    'half_window2': self.leg_reluctances['right'] + self.parallel(self.leg_reluctances['center'],self.leg_reluctances['left']),
                                    'full'        : self.leg_reluctances['center']+ self.parallel(self.leg_reluctances['left'],self.leg_reluctances['right'])}
    
        #create dictionary of parallel reluctances.  Used in superposition calculation
        self.parallel_reluctances = {'Rpc2': self.parallel(self.leg_reluctances['center'],self.leg_reluctances['right']),
                                     'Rpc1': self.parallel(self.leg_reluctances['center'],self.leg_reluctances['left']),
                                     'Rp12': self.parallel(self.leg_reluctances['left'],  self.leg_reluctances['right'])}
        
        
        
    def reluctance(self,ur,length,area): 
        uo = 4*np.pi*1e-7
        return length/(uo*ur*area)
      
    def total_reluctance(self,segmentlist):
        reluctance_list = [self.segment_reluctances[segment] for segment in segmentlist]
        return sum(reluctance_list)
    
    def parallel(self,a,b):
        return a*b/(a+b)

#### Permeance

In [0]:
#class Permeance_model:
# use inheritance to pull in traits from reluctance model  
#    self.leg_permeances = self.permeance_dicts(self.leg_reluctances)  
#    def permeance_dicts(self,r):
#        return {key: 1/value for key, value in r.items()}


#### DC fluxes and currents

In [0]:
class DC_fluxes_and_currents:
  
    def __init__(self, core, winding, iout):  #warning: an imbalance in leg currents could produce DC flux in the magnetizing 
        self.core=core               
        self.winding=winding
        self.iout = iout
        self.reluctances=Reluctance_model(self.core)
        
        self.dc_fluxes_and_currents=self.calculate_dc_fluxes_and_currents()
        self.dc_fluxes = self.dc_fluxes_and_currents['dc_flux_per_leg']
        self.dc_currents = self.dc_fluxes_and_currents['amps_per_winding']
        
    def calculate_dc_fluxes_and_currents(self):
        nc1=self.winding.turns['full1']
        nc2=self.winding.turns['full2']
        n1=self.winding.turns['half_window1']
        n2=self.winding.turns['half_window2']
        
        r1=self.reluctances.leg_reluctances['left']
        r2=self.reluctances.leg_reluctances['right']
        rc=self.reluctances.leg_reluctances['center']
        
        rpc1=self.reluctances.parallel_reluctances['Rpc1']
        rpc2=self.reluctances.parallel_reluctances['Rpc2']
        rp12=self.reluctances.parallel_reluctances['Rp12']
        
        
        coefficients = np.array([
            [n1, n2, -r1, -r2],
            [n1, n2,   0,   0],
            [nc1,-nc2,  0,  0],
            [n1*rpc2/(r1+rpc2)-rp12*nc1/(rc+rp12)-n1, rp12*nc2/(rc+rp12)-rpc1*n2/(r2+rpc1), r1, 0]])
        
        const_array = np.array([        0,
                                self.iout,
                                        0,
                                        0])
        
        #solve for [I1,I2,flux1,flux2]
        solution = np.linalg.solve(coefficients,const_array)
        
        #half_window1 winding is in series with full1 winding, so both are I1
        #half_window2 winding is in series with full2 winding, so both are I2
        amps_per_winding = {'half_window1':solution[0], 'full1':solution[0],
                            'half_window2':solution[1], 'full2':solution[1]}
        
        #'up' = positive for all 3 legs
        dc_flux_per_leg = {'left':solution[2],'right':-solution[3],'center':-(solution[2]-solution[3])}
        
        return {'amps_per_winding':amps_per_winding, 'dc_flux_per_leg':dc_flux_per_leg}


#### AC fluxes

In [0]:
class AC_fluxes:   #only for Vout=<Vin/4
    
    def __init__(self, core, winding, vin, vout, fs, state):
      
        self.core=core
        
        self.winding=winding        
        self.fullturns1  = self.winding.turns['full1']
        self.fullturns2  = self.winding.turns['full2']
        self.halfturns1 = self.winding.turns['half_window1']   # number of half turns through window 1
        self.halfturns2 = self.winding.turns['half_window2']   # number of half turns through window 2
        
        self.vin = vin
        self.vout = vout
        self.duty = self.vout/(self.vin/2)
        self.fs = fs
        
        self.state = state
        
    def fluxrates(self):
        vphase1 = {'state1':self.vin/2,
                   'state2':0,
                   'state3':0,
                   'state4':0}
        vphase2 = {'state1':0,
                   'state2':0,
                   'state3':self.vin/2,
                   'state4':0}
        
        
        #3 equations, 3 unknowns to solve for flux rates.  
        #'up' = positive for all 3 legs
        coefficients = np.array([
                        [self.halfturns1, -self.fullturns1,               0],
                        [              0, -self.fullturns2, self.halfturns2],
                        [              1,             1,               1]])
        
        winding_voltages = np.array([vphase1[self.state]-self.vout,
                                     self.vout-vphase2[self.state],
                                                                 0])
        legs = ['left','center','right']
        fluxratedict=dict(zip(legs,np.linalg.solve(coefficients,winding_voltages)))
        return  fluxratedict
      

    def time_states(self):
        ts = 1/self.fs
        t1 = self.duty*ts
        t3 = t1
        t2 = (ts-t1-t3)/2
        t4 = t2
        return {'state1':t1,
                'state2':t2,
                'state3':t3,
                'state4':t4}
        
    def bac(self):
        flxrates=self.fluxrates()
        tstate=self.time_states()[self.state]
        b_pp={leg:fluxrate*tstate/self.core.leg_areas[leg] for leg, fluxrate in flxrates.items() }
        return b_pp

#### AC currents

#### DC+AC Winding currents

#### Flux Densities

###Input Parameters

In [0]:
Vin = 48
Vout = 12
iout = 60

area=62e-6
fs=200e3

state='state1'

PQ26 = {'length':32e-3, 'height':11e-3, 'Area':119e-6}  #length, height  at core center (mean flux path).  Area is at center post
nturns = {'half_window1':1, 'full1':3, 'full2':3, 'half_window2':1}
gap_lengths = {'AF':0.3e-3, 'BE':0.3e-3, 'CD':0.3e-3}

pq26intcore=PQcore(PQ26,3200,gap_lengths)
wind3p5turn=Winding(nturns)

###Results

In [31]:

reluctances = Reluctance_model(pq26intcore)
#reluctances.leg_reluctances,reluctances.parallel_reluctances
1/reluctances.leg_reluctances['left'],9/reluctances.leg_reluctances['center']

(2.3861942405245705e-07, 4.436743110078475e-06)

In [32]:
dc_fluxes_and_currents=DC_fluxes_and_currents(core=pq26intcore,winding=wind3p5turn,iout=iout)
#dc_fluxes_and_currents.dc_fluxes['full1']#/(PQ26['Area']/2)
dc_fluxes_and_currents.dc_fluxes#['left']/(PQ26['Area']/2)

{'center': -0.0,
 'left': 7.1585827215737105e-06,
 'right': -7.1585827215737105e-06}

In [33]:
ac_rate_state1= AC_fluxes(core=PQcore(PQ26,3200,gap_lengths), winding=Winding(nturns), vin=Vin, vout=Vout, fs=fs, state=state)
#ac_rate_state1.bac()
ac_rate_state1.fluxrates()

{'center': -3.4285714285714284,
 'left': 1.7142857142857153,
 'right': 1.7142857142857142}